In [1]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np

df = pd.read_csv('patients_data_from_images.csv')
df['label'] = (df['total_md_gap_per_year']<-2).astype(int)

X = df.drop(['total_md_gap_per_year', 'label'], axis = 1)
Y = df['label']

scaler = MinMaxScaler()
X = X.fillna(0)
X = scaler.fit_transform(X)
Y = np.array(Y)

print(X.shape, Y.shape)

(101, 118) (101,)


# FullBN (5) -- meta/aux input at start

In [2]:
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.layers import concatenate, Dropout, Flatten, Dense, Input, Conv2D, MaxPooling2D, BatchNormalization, Activation
from keras.optimizers import SGD
from keras.models import Model, Sequential
import numpy as np
import pandas as pd
import keras
import math


# define parameters
input_tensor_shape = (118,)
epochs = 500

#####################################################################################################
######################################          Model          ######################################
#####################################################################################################

# Define Input
inputs = Input(shape=input_tensor_shape)

# FullBN (3) - (64, 32, 16)
x = Dense(256, init = 'glorot_normal')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Dense(128)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Dense(64)(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Dense(32)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Dense(16)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

output = Dense(1, activation = 'sigmoid')(x)

# compile model, summarize layers & plot graph
model = Model(inputs=inputs, outputs=output)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

#####################################################################################################
#####################################################################################################
#####################################################################################################

# define callbacks
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=1, write_graph=True, write_grads=True, write_images=False)

model.fit(X, Y, epochs = epochs, validation_split = 0.8, class_weight = {1:3.625, 0:1.0}, callbacks = [early], verbose = 1)  

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(256, kernel_initializer="glorot_normal")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 118)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                7616      
_________________________________________________________________
batch_normalization_3 (Batch (None, 64)                256       
_________________________________________________________________
activation_3 (Activation)    (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                2080      
_________________________________________________________________
batch_normalization_4 (Batch (None, 32)                128       
_________________________________________________________________
activation_4 (Activation)    (None, 32)                0         
__________

# Conv2D (6) - aux input at end

In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input, Conv2D, MaxPooling2D, concatenate
from keras.models import Model
from keras import backend as K
import numpy as np
import pandas as pd
import keras
import math

# define parameters
# define parameters
img_width, img_height = 6, 6
input_shape = (1, img_width, img_height)
epochs = 500

#####################################################################################################
######################################          Model          ######################################
#####################################################################################################

inputs = Input(shape = (784, 60, 20))

x = Conv2D(32, (3,3), input_shape = input_shape)(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(64, (3,3))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(128, (3,3))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv2D(256, (3,3))(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Flatten()(x)
#merged = concatenate([meta_input, flat1])
x = Dense(1024, activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(512, activation = 'relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation = 'sigmoid')(x)

# compile model, summarize layers & plot graph
model = Model(inputs=inputs, outputs=output)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

#####################################################################################################
#####################################################################################################

# define callbacks
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_grads=True, write_images=False)

model.fit([X1, X2], Y, epochs = epochs, validation_split = 0.8, class_weight = , callbacks = [tbCallBack, early])

# Inception (6) - aux input at end

In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, Input, Conv2D, MaxPooling2D, concatenate
from keras.models import Model
from keras import backend as K
import numpy as np
import pandas as pd
import keras
import math

# define parameters
# define parameters
img_width, img_height = 6, 6
input_shape = (1, img_width, img_height)
epochs = 500

#####################################################################################################
######################################          Model          ######################################
#####################################################################################################

input_img = Input(shape=(256, 256, 3))

tower_1 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(tower_1)

tower_2 = Conv2D(64, (1, 1), padding='same', activation='relu')(input_img)
tower_2 = Conv2D(64, (5, 5), padding='same', activation='relu')(tower_2)

tower_3 = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_img)
tower_3 = Conv2D(64, (1, 1), padding='same', activation='relu')(tower_3)

# combine towers, flatten and concat
combined = concatenate([tower_1, tower_2, tower_3], axis=1)
flat = Flatten()(combined)
merged = concatenate([meta_data, flat])

# last FCN
x = Dense(1024, activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(152, activation = 'relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation = 'sigmoid')(x)

# compile model, summarize layers & plot graph
model = Model(inputs=inputs, outputs=output)
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

#####################################################################################################
#####################################################################################################

# define callbacks
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_grads=True, write_images=False)

model.fit([X1, X2], Y, epochs = epochs, validation_split = 0.8, class_weight = , callbacks = [tbCallBack, early])

In [9]:
import pandas as pd
import numpy as np

# reading in data files
df = pd.read_csv('Glaucoma metadata V1.csv')
df.drop('Unnamed: 0', axis = 1, inplace = True)
df2 = pd.read_csv('patients_data_from_images.csv')

# converting datetimes
df['Date'] =  pd.to_datetime(df['Date'], format='%Y-%m-%d')
df2['date_1'] =  pd.to_datetime(df2['date_1'], format='%Y%m%d')

# processing meta data
df.sort_values(['Patient ID', 'Date'], inplace = True)
df = df[df['Right or Left'] == 0]
df.drop_duplicates(['Patient ID', 'Date'], axis = 1, keep = 'first', inplace = True)
df.reset_index(drop = True, inplace = True)
df.to_csv('temp.csv', index = False)
# processesed outside in excel to only keep first 2 record for each patient -- reimported
df = pd.read_csv('temp2.csv', encoding='cp1252')
df01 = df[df['Occurrence'] == 1]
df02 = df[df['Occurrence'] == 2]
df03 = df02[['Patient ID', 'VFI', 'MD', 'PSD', 'FL', 'FP', 'FN', 'GHT', 'BCVA', 'CDR','IOP']]
df = pd.merge(df01, df03, on = 'Patient ID')
df.to_csv('temp3.csv')


# import and merge
df = pd.read_csv('temp3.csv')
df3 = pd.merge(df, df2, left_on = 'Patient ID', right_on = 'patient_id')
df3.drop(['Patient ID', 'date_1', 'Duration', 'MD gradient'] , axis = 1, inplace = True)
df3['MD Drop Fast'].replace({'True': 1, 'False': 0}, inplace = True)
df3 = df3.applymap(lambda x: 1 if x == True else x)
df3 = df3.applymap(lambda x: 0 if x == False else x)

"\n# merging\ndf3 = pd.merge(df, df2, left_on = 'Patient ID', right_on = 'patient_id')"

In [ ]:
# Imports
import numpy as np
import tensorflow as tf

#need to define label 

# Input Layer
input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

# 1st Dense Layer
x = tf.layers.dense(input_layer, 256, kernel_initializer = tf.glorot_normal_initializer())
x = tf.layers.batch_normalization(x)
x = tf.nn.relu(x, 'relu')

# 2nd Dense Layer
x = tf.layers.dense(input_layer, 128)
x = tf.layers.batch_normalization(x)
x = tf.nn.relu(x, 'relu')

# 3rd Dense Layer
x = tf.layers.dense(input_layer, 64)
x = tf.layers.batch_normalization(x)
x = tf.nn.relu(x, 'relu')

# 4th Dense Layer
x = tf.layers.dense(input_layer, 32)
x = tf.layers.batch_normalization(x)
x = tf.nn.relu(x, 'relu')

# 5th Dense Layer
x = tf.layers.dense(input_layer, units=16)
x = tf.layers.batch_normalization(x)
x = tf.nn.relu(x, 'relu')

# dropout layer
x = tf.layers.dropout(x, rate = 0.2)

# logits layer
logits = tf.layers.dense(x, 2)

predictions = {
  # Generate predictions (for PREDICT and EVAL mode)
  "classes": tf.argmax(logits, axis=1),
  # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
  # `logging_hook`.
  "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
}

# Calculate Loss (for both TRAIN and EVAL modes)
loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

# Configure the Training Op (for TRAIN mode)
if mode == tf.estimator.ModeKeys.TRAIN:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(
    loss=loss,
    global_step=tf.train.get_global_step())
return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

# Add evaluation metrics (for EVAL mode)
eval_metric_ops = {
  "accuracy": tf.metrics.accuracy(
      labels=labels, predictions=predictions["classes"])}
return tf.estimator.EstimatorSpec(
  mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [80]:
X2[1]

array([[2, 2, 2],
       [2, 2, 2],
       [2, 2, 2]])

In [82]:
np.ndarray.flatten(X2[0])

array([1, 1, 1, 1, 1, 1, 1, 1, 1])

In [75]:
for index, row in X1.iterrows():
    print(np.concatenate((row.values,np.ndarray.flatten(X2))))

[4 5 3 4 5 3 1 2 5 9 2 8]
[1 2 5 4 5 3 1 2 5 9 2 8]
[9 2 8 4 5 3 1 2 5 9 2 8]


In [88]:
arr_list = list()

for i in range(len(X1)):
    arr = np.concatenate((X1[i].values,np.ndarray.flatten(X2[i])))
    arr_list.append(arr)

In [89]:
arr_list

[array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64),
 array([1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64),
 array([1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int64)]